In [4]:
%load_ext autoreload
%autoreload 2
import sys

sys.path.append("../src/")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
from mcr_py.package.logger import Timed, rlog, setup

setup("INFO")


import os

import folium
import pandas as pd
from mcr_py.package import storage, strtime
from mcr_py.package.geometa import GeoMeta
from mcr_py.package.osm import osm

In [6]:
city_id = "Koeln"
stops_path = "../data/cleaned/stops.csv"
osm_path = osm.get_osm_path_from_city_id(city_id)


geo_meta = GeoMeta.load("../data/geometa.pkl")

with Timed.info("Reading stops"):
    other_stops_df = storage.read_gdf(stops_path)

if not os.path.exists(osm_path) and city_id:
    rlog.info("Downloading OSM data")
    osm.download_city(city_id, osm_path)
else:
    rlog.info("Using existing OSM data")

osm_reader = osm.new_osm_reader(osm_path)

with Timed.info("Getting OSM graph"):
    nodes, edges = osm.get_graph_for_city_cropped_to_boundary(
        osm_reader, geo_meta, "walking"
    )

[09:26:29] INFO     Reading stops                                                                   ]8;id=948186;file:///tmp/ipykernel_100819/3959691737.py\3959691737.py]8;;\:]8;id=50742;file:///tmp/ipykernel_100819/3959691737.py#8\8]8;;\

           INFO     Reading stops done (0.66 seconds)                                               ]8;id=732917;file:///tmp/ipykernel_100819/3959691737.py\3959691737.py]8;;\:]8;id=164201;file:///tmp/ipykernel_100819/3959691737.py#8\8]8;;\

           INFO     Using existing OSM data                                                        ]8;id=506253;file:///tmp/ipykernel_100819/3959691737.py\3959691737.py]8;;\:]8;id=121251;file:///tmp/ipykernel_100819/3959691737.py#15\15]8;;\

           INFO     Getting OSM graph                                                              ]8;id=290725;file:///tmp/ipykernel_100819/3959691737.py\3959691737.py]8;;\:]8;id=756959;file:///tmp/ipykernel_100819/3959691737.py#19\19]8;;\

           INFO     Loading OSM network from cache (walking)                                       ]8;id=318611;file:///tmp/ipykernel_100819/3959691737.py\3959691737.py]8;;\:]8;id=26809;file:///tmp/ipykernel_100819/3959691737.py#19\19]8;;\

[09:26:35] INFO     Getting OSM graph done (6.36 seconds)                                          ]8;id=195634;file:///tmp/ipykernel_100819/3959691737.py\3959691737.py]8;;\:]8;id=404765;file:///tmp/ipykernel_100819/3959691737.py#19\19]8;;\

In [1]:
path_source = "/home/moritz/dev/uni/mcr-py/data/mcr.pkl"

data = storage.read_any_dict(path_source)
path_manager = data["path_manager"]
# node_map = data["multi_modal_node_to_resetted_map"]
# walking_node_map = data["walking_node_to_resetted_map"]
# reverse_node_map = {v: k for k, v in node_map.items()}
# reverse_walking_node_map = {v: k for k, v in walking_node_map.items()}
# stops_df = data["stops_df"]
bags_i = data["bags_i"]

NameError: name 'storage' is not defined

In [2]:
# translation_data = storage.read_any_dict("../data/car_step_translations.pkl")
# reverse_node_map = translation_data["resetted_to_multi_modal_node_map"]
# node_map = translation_data["multi_modal_node_to_resetted_map"]

In [10]:
labels = pd.DataFrame(
    [
        (label.node_id, label.values[0], label.values[1], n_transfers, label)
        for n_transfers, bags in bags_i.items()
        for bag in bags.values()
        for label in bag
    ],
    columns=["osm_node_id", "time", "cost", "n_transfers", "label"],
)

labels["human_readable_time"] = labels["time"].apply(strtime.seconds_to_str_time)
labels = labels.sort_values("n_transfers")
labels = labels.drop_duplicates(subset=["osm_node_id", "time", "cost"], keep="first")
labels.drop(columns=["label"])

,osm_node_id,time,cost,n_transfers,human_readable_time
0,1868389085,29044,0,0,08:04:04
1745,6662781854,29329,0,0,08:08:49
1746,9709744522,28988,0,0,08:03:08
1747,3674115309,29444,0,0,08:10:44
1748,1877329601,29023,0,0,08:03:43
...,...,...,...,...,...
4703,7890873890,29062,100,1,08:04:22
4704,7170430314,29545,0,1,08:12:25
4705,7170430314,29060,100,1,08:04:20
4706,2831639993,28928,100,1,08:02:08


In [11]:
from mcr_py.package.minute_city import minute_city

In [12]:
pois = minute_city.fetch_pois_for_area(geo_meta.boundary, nodes)

In [24]:
labels["target_id_osm"] = labels["osm_node_id"]
labels["start_id_hex"] = "aaa"

In [25]:
poi_labels = minute_city.add_pois_to_labels(labels, pois)

In [34]:
poi_labels

,osm_node_id,time,cost,n_transfers,label,human_readable_time,target_id_osm,start_id_hex,nearest_osm_node_id,grocery,education,health,banks,parks,sustenance,shops
0,6040204998,29404,0,0,"IntermediateLabel(values=[29404, 0], hidden_va...",08:10:04,6040204998,aaa,6040204998,0,0,0,0,0,1,0
1,6040204998,29045,100,1,"IntermediateLabel(values=[29045, 100], hidden_...",08:04:05,6040204998,aaa,6040204998,0,0,0,0,0,1,0
2,6040204998,29460,0,1,"IntermediateLabel(values=[29460, 0], hidden_va...",08:11:00,6040204998,aaa,6040204998,0,0,0,0,0,1,0
3,2741627716,29320,0,0,"IntermediateLabel(values=[29320, 0], hidden_va...",08:08:40,2741627716,aaa,2741627716,1,0,0,0,0,0,0
4,2741627716,29320,0,0,"IntermediateLabel(values=[29320, 0], hidden_va...",08:08:40,2741627716,aaa,2741627716,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,6038251142,29048,0,0,"IntermediateLabel(values=[29048, 0], hidden_va...",08:04:08,6038251142,aaa,6038251142,0,0,0,0,0,0,1
92,6038251142,29104,0,1,"IntermediateLabel(values=[29104, 0], hidden_va...",08:05:04,6038251142,aaa,6038251142,0,0,0,0,0,0,1
93,6038251142,28914,100,1,"IntermediateLabel(values=[28914, 100], hidden_...",08:01:54,6038251142,aaa,6038251142,0,0,0,0,0,0,1
94,3829822082,29039,100,1,"IntermediateLabel(values=[29039, 100], hidden_...",08:03:59,3829822082,aaa,3829822082,1,0,0,0,0,0,0


In [16]:
# types = ["grocery", "education", "health", "banks", "parks", "sustenance", "shops"]
types = list(pois["type"].unique())

In [17]:
profiles_df = minute_city.get_profiles_df(poi_labels, types)

[09:28:39] INFO     Grouping labels                                                               ]8;id=646175;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/minute_city/minute_city.py\minute_city.py]8;;\:]8;id=817080;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/minute_city/minute_city.py#73\73]8;;\

           INFO     Grouping labels done (0.01 seconds)                                           ]8;id=216456;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/minute_city/minute_city.py\minute_city.py]8;;\:]8;id=118933;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/minute_city/minute_city.py#73\73]8;;\

           INFO     Calculating profiles                                                          ]8;id=129652;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/minute_city/minute_city.py\minute_city.py]8;;\:]8;id=639498;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/minute_city/minute_city.py#81\81]8;;\

  0%|          | 0/1 [00:00<?, ?it/s]

           INFO     Calculating profiles done (0.57 seconds)                                      ]8;id=515576;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/minute_city/minute_city.py\minute_city.py]8;;\:]8;id=123154;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/minute_city/minute_city.py#81\81]8;;\

           INFO     Creating profiles dataframe                                                   ]8;id=951783;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/minute_city/minute_city.py\minute_city.py]8;;\:]8;id=846817;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/minute_city/minute_city.py#92\92]8;;\

           INFO     Creating profiles dataframe done (0.02 seconds)                               ]8;id=307306;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/minute_city/minute_city.py\minute_city.py]8;;\:]8;id=208523;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/minute_city/minute_city.py#92\92]8;;\

In [18]:
profiles_df

,cost_0,cost_100,any_column_different,required_cost_for_optimal,optimal
hex_id,,,,,
aaa,796.0,316.0,True,100,316.0


In [19]:
max_time = strtime.str_time_to_seconds("09:00:00")
max_cost = 1000
max_transfers = 10

valid_labels = labels[
    (labels["time"] <= max_time)
    & (labels["cost"] <= max_cost)
    & (labels["n_transfers"] <= max_transfers)
]

In [20]:
# selection = valid_labels.sample(min(1000, len(valid_labels)))

max_cost = valid_labels.cost.max()

from colorsys import hls_to_rgb
from typing import Dict


def hsl_to_hex(h: float, l: float, s: float) -> str:
    r, g, b = [int(x * 255.0) for x in hls_to_rgb(h, l, s)]
    return "#{:02x}{:02x}{:02x}".format(r, g, b)


def generate_colors_hsl(max_cost: float) -> Dict[int, str]:
    if max_cost == 0:
        return {0: "#ffcccc"}
    return {
        cost: hsl_to_hex(h=0, l=0.5 + 0.4 * (max_cost - cost) / max_cost, s=0.8)
        for cost in range(int(max_cost) + 1)
    }


colors = generate_colors_hsl(max_cost)

In [21]:
start_node_id = labels[labels.time == labels.time.min()].iloc[0].osm_node_id

In [ ]:
nodes_by_id = nodes.set_index("id")
nodes_by_id["id"] = nodes_by_id.index

start_node = nodes_by_id.loc[start_node_id]

m = folium.Map(location=[start_node.lat, start_node.lon], zoom_start=13)

# for label in selection.itertuples():
for label in labels.itertuples():
    node = nodes_by_id.loc[label.osm_node_id]
    folium.CircleMarker(
        location=[node.lat, node.lon],
        radius=3,
        weight=1,
        fill=True,
        fill_color=colors[label.cost],
        fill_opacity=1,
        color="black",
        popup=f"{label.human_readable_time} ({label.cost})",
    ).add_to(m)

folium.CircleMarker(
    location=[start_node.lat, start_node.lon],
    radius=5,
    weight=1,
    fill=True,
    fill_color="green",
    fill_opacity=1,
    color="black",
    popup="Start",
).add_to(m)

m

In [19]:
from mcr_py.package.mcr.path import GTFSPath, Path, PathType

translator_map = {
    # PathType.WALKING: reverse_walking_node_map,
    # PathType.CYCLING_WALKING: reverse_node_map,
    PathType.DRIVING_WALKING: reverse_node_map,
    # PathType.PUBLIC_TRANSPORT: None,
}
no_prefix_reverse_node_map = {k: int(v[1:]) for k, v in reverse_node_map.items()}
no_prefix_reverse_node_map = {k: int(v[1:]) for k, v in reverse_node_map.items()}
no_prefix_translator_map = {
    PathType.DRIVING_WALKING: no_prefix_reverse_node_map,
    PathType.PUBLIC_TRANSPORT: None,
}

In [20]:
result_bags = bags_i[1]

In [21]:
walking_result_bags_flat = []
for node_id, bag in result_bags.items():
    for label in bag:
        walking_result_bags_flat.append((node_id, label))

In [22]:
path_objs_with_ids = pd.Series(
    list(
        map(
            lambda x: (
                x[0],
                path_manager.reconstruct_and_translate_path_for_label(
                    x[1], no_prefix_translator_map
                ),
            ),
            walking_result_bags_flat[:10000],
        )
    )
)
node_ids, path_objs = list(zip(*path_objs_with_ids))
path_objs[:3]

KeyError: <PathType.WALKING: 'walking'>

In [ ]:
path_objs[0]

[Path(path_type=PathType.DRIVING_WALKING, path=[394001227, 1679917802, 4394977417, 2694582111, 4297860764, 266709751, 8805331129, 737784943, 266709711, 626052074, 3141997563, 737845925, 266710405, 2427289544, 4297860767, 2116624890, 3141997564, 276278173, 734098359, 2285985235, 279486552, 280124803, 943958005, 8789517655, 943957929, 671213002, 733934973, 315117901, 671213003, 2116634417, 26110723, 627998733, 3190145352, 406937317, 8789517653, 2116641930, 2116641928, 2427287508, 279166995, 5338775614, 8789517656, 5338775615, 2116641932, 2427289541, 3190145355, 279167057, 5338775612, 667027425, 1679918212, 2116645220, 279166998, 3447314049, 279167348, 8254554431, 8254554432, 3190145357, 8254554430, 2427289542, 279167346, 8254554429, 279166996, 2159999733, 1679918259, 315118775, 8254554447, 1462694008, 279231485, 3190145358, 395576169, 1679918288, 315119693, 8254554484, 654454038, 661087237, 2645720096, 654454041, 8254554491, 8254554506, 8254554524, 8254554518, 654454101, 3447307341, 6544

In [ ]:
reverse_node_map[35781]

'D394001227'

In [ ]:
labels.drop(columns=["label"])

,osm_node_id,time,cost,n_transfers,human_readable_time,target_id_osm,start_id_hex
0,8254554550,28876,38,0,08:01:16,8254554550,aaa
3279,266712495,28830,19,0,08:00:30,266712495,aaa
3278,266712495,29227,0,0,08:07:07,266712495,aaa
3277,3734989924,28864,38,0,08:01:04,3734989924,aaa
3276,3734989924,28928,19,0,08:02:08,3734989924,aaa
...,...,...,...,...,...,...,...
1635,8634843176,29017,0,0,08:03:37,8634843176,aaa
1634,5375654928,28837,19,0,08:00:37,5375654928,aaa
1633,8541426514,28827,19,0,08:00:27,8541426514,aaa
1639,1539888990,28833,19,0,08:00:33,1539888990,aaa


In [ ]:
import random

i = random.sample(list(labels.osm_node_id.unique()), 1)[0]
# i = 3922625314
print(i)
selection = labels[labels.osm_node_id == i]
selection.drop(columns=["label"])

1679798523


,osm_node_id,time,cost,n_transfers,human_readable_time,target_id_osm,start_id_hex
2924,1679798523,28856,19,0,08:00:56,1679798523,aaa


In [ ]:
start_time = strtime.str_time_to_seconds("08:00:00")

In [ ]:
def format_meta(meta, previous_meta):
    values = meta["values"]
    arrival_time = values[0]
    cost = values[1]

    if previous_meta:
        previous_values = previous_meta["values"]
        previous_arrival_time = previous_values[0]
        previous_cost = previous_values[1]

        arrival_time -= previous_arrival_time
        cost -= previous_cost
    else:
        arrival_time -= start_time

    return f"{strtime.seconds_to_str_time(arrival_time)} ({cost})"

In [ ]:
toloop = selection

# stops_by_id = stops_df.set_index("stop_id")
sample_label = selection.iloc[0]
sample_node_id = sample_label.osm_node_id
nodes_by_id = nodes.set_index("id")
nodes_by_id["id"] = nodes_by_id.index
sample_node = nodes_by_id.loc[sample_node_id]

m = folium.Map(location=[sample_node.lat, sample_node.lon], zoom_start=13)

for row in toloop.itertuples():
    label = row.label
    end_node_id = row.osm_node_id
    end_node = nodes_by_id.loc[end_node_id]

    folium.CircleMarker(
        location=[end_node.lat, end_node.lon],
        popup=f"End: {end_node_id}",
        color="red",
        radius=3,
    ).add_to(m)

    paths = path_manager.reconstruct_and_translate_path_for_label(label, translator_map)
    for i, path in enumerate(paths):
        if isinstance(path, Path):
            if path.path == []:
                continue
            cycling_path_nodes = [
                nodes_by_id.loc[int(node_id[1:])]
                for node_id in path.path
                if node_id[0] == "D"
            ]
            walking_path_nodes = [
                nodes_by_id.loc[int(node_id[1:])]
                for node_id in path.path
                if node_id[0] == "W"
            ]
            path_lat_lon = [(node.lat, node.lon) for node in cycling_path_nodes]
            previous_meta = paths[i - 1].meta if i > 0 else None
            meta = format_meta(path.meta, previous_meta)
            if path_lat_lon != []:
                folium.PolyLine(
                    path_lat_lon, color="blue", weight=2, popup=str(meta)
                ).add_to(m)
            path_lat_lon = [(node.lat, node.lon) for node in walking_path_nodes]
            if path_lat_lon != []:
                folium.PolyLine(
                    path_lat_lon, color="red", weight=2, popup=str(meta)
                ).add_to(m)
        elif isinstance(path, GTFSPath):
            raise Exception("GTFSPath not supported")
            start_stop_id = path.start_stop_id
            end_stop_id = path.end_stop_id
            start_stop = stops_by_id.loc[start_stop_id]
            end_stop = stops_by_id.loc[end_stop_id]
            trip = path.trip_id
            if len(trip) >= 10:
                trip = trip[:10] + "..."

            previous_meta = paths[i - 1].meta if i > 0 else None
            line_msg = f"Trip: {trip}\n---\n {format_meta(path.meta, previous_meta)}"

            path_lat_lon = [
                (float(start_stop.stop_lat), float(start_stop.stop_lon)),
                (float(end_stop.stop_lat), float(end_stop.stop_lon)),
            ]
            folium.PolyLine(
                path_lat_lon,
                color="green",
                weight=2,
                popup=line_msg,
            ).add_to(m)

            folium.CircleMarker(
                location=[float(start_stop.stop_lat), float(start_stop.stop_lon)],
                popup=f"Start: {start_stop.stop_name}",
                color="green",
                radius=3,
            ).add_to(m)
            folium.CircleMarker(
                location=[float(end_stop.stop_lat), float(end_stop.stop_lon)],
                popup=f"End: {end_stop.stop_name}",
                color="green",
                radius=3,
            ).add_to(m)
        else:
            raise Exception("Unknown path type")

m